In [1]:
#| default_exp 24_msmarco-hard-negatives-systems

In [2]:
%reload_ext autoreload
%autoreload 2

In [44]:
#| export
import pickle, scipy.sparse as sp, numpy as np, argparse, os, json
from xclib.utils.sparse import retain_topk
from tqdm.auto import tqdm
from typing import Dict

from sugar.core import *

## Setup

In [7]:
fname = '/Users/suchith720/Projects/data/msmarco-hard-negatives.jsonl'

In [38]:
#| export
def load_negatives_system_info(fname:str):
    negatives, systems = {}, {}
    with open(fname) as file:
        for line in file:
            c = json.loads(line)
            for k,v in c['neg'].items():
                sys_id = systems.setdefault(k, len(systems))
                negatives[c['qid']] = {o:sys_id for o in v}
    return systems, negatives
                

In [29]:
%%time
systems, negatives = load_negatives_system_info(fname)

CPU times: user 1min 1s, sys: 1.53 s, total: 1min 2s
Wall time: 1min 2s


In [31]:
len(systems)

13

In [45]:
#| export
def create_system_matrix(data_meta:sp.csr_matrix, data_info:Dict, meta_idx2id:Dict):
    data = list()
    for i, (p,q) in tqdm(enumerate(zip(data_meta.indptr, data_meta.indptr[1:])), total=data_meta.shape[0]):
        negs = negatives[int(data_info[i])]
        data.extend([negs[int(meta_idx2id[j])] for j in data_meta.indices[p:q]])
    return sp.csr_matrix((data, data_meta.indices, data_meta.indptr), dtype=np.int64, shape=data_meta.shape)


In [46]:
#| export
def get_topk_scores_per_system(data_meta:sp.csr_matrix, systems:Dict, data_sys:sp.csr_matrix, topk:int=5):
    scores = sp.csr_matrix(data_meta.shape)
    for k,v in systems.items():
        mask = data_sys.copy()
        mask.data[:] = np.where(mask.data == v, 1, 0)
        mask.eliminate_zeros()
        scores += retain_topk(data_meta.multiply(mask), k=topk)
    return scores
    

## Driver

In [41]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_meta', type=str, required=True)
    parser.add_argument('--data_info', type=str, required=True)
    parser.add_argument('--meta_info', type=str, required=True)
    parser.add_argument('--negatives', type=str, required=True)
    parser.add_argument('--topk', type=int, default=5)
    
    return parser.parse_args()
    

In [47]:
data_meta_file = "/data/datasets/msmarco/XC//ce-negatives_trn_X_Y.npz"
data_info_file = "/data/datasets/msmarco/XC//raw_data/train.raw.txt"
meta_info_file =  "/data/datasets/msmarco/XC//raw_data/ce-scores.raw.txt"

neg_file = "/data/datasets/msmarco/ce_scores/msmarco-hard-negatives.jsonl"

topk = 5

In [48]:
#| export
if __name__ == '__main__':
    input_args = parse_args()
    
    data_meta = sp.load_npz(input_args.data_meta)
    data_info, _ = load_raw_file(input_args.data_info)
    meta_info, _ = load_raw_file(input_args.meta_info)

    systems, negatives = load_negatives_system_info(input_args.negatives)

    meta_idx2id = {k:v for k,v in enumerate(meta_info)}
    data_sys = create_system_matrix(data_meta, data_info, meta_idx2id)

    scores = get_topk_scores_per_system(data_meta, systems, data_sys, topk=input_args.topk)

    file_parts = data_meta_file.split('_', maxsplit=1)
    out_file = file_parts[0] + f'-topk-{topk:02d}_' + file_parts[1]
    sp.save_npz(out_file, scores)
    

usage: ipykernel_launcher.py [-h] --data_meta DATA_META --data_info DATA_INFO
                             --meta_info META_INFO --negatives NEGATIVES
                             [--topk TOPK]
ipykernel_launcher.py: error: the following arguments are required: --data_meta, --data_info, --meta_info, --negatives


SystemExit: 2

/Users/suchith720/miniconda3/envs/mogic/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
